In [1]:
!pip install unsloth torch accelerate transformers trl datasets bitsandbytes -q

from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import DPOConfig, DPOTrainer  # Direct Preference Optimization (or similar)
import torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.8/348.8 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 130.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 20.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavi

In [2]:
model_name = "unsloth/smollm2-135m"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto"
)

==((====))==  Unsloth 2025.11.1: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/742 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

## Dataset

In [3]:
!pip install datasets -q

In [4]:
from datasets import load_dataset, DatasetDict

In [5]:
DATASET_CHOICE = "Dahoas/full-hh-rlhf"   # <-- change this line if you want another dataset

def load_preference_dataset(name: str, split: str = "train"):
    """
    Returns a Dataset with the exact columns:
      - prompt (str)
      - chosen (str)
      - rejected (str)
    """
    if name == "Dahoas/full-hh-rlhf":
        # Already in (prompt, chosen, rejected) format
        ds = load_dataset("Dahoas/full-hh-rlhf", split=split)
        # Keep only needed columns
        keep = {"prompt", "chosen", "rejected"}
        drop_cols = [c for c in ds.column_names if c not in keep]
        if drop_cols:
            ds = ds.remove_columns(drop_cols)
        return ds

    if name == "Dahoas/synthetic-instruct-gptj-pairwise":
        # Already in (prompt, chosen, rejected) format
        ds = load_dataset("Dahoas/synthetic-instruct-gptj-pairwise", split=split)
        keep = {"prompt", "chosen", "rejected"}
        drop_cols = [c for c in ds.column_names if c not in keep]
        if drop_cols:
            ds = ds.remove_columns(drop_cols)
        return ds

    if name == "Anthropic/hh-rlhf":
        # This dataset contains 'chosen' and 'rejected' as full dialogues (no explicit 'prompt').
        # We'll synthesize a minimal prompt so downstream code expecting a 'prompt' column works.
        raw = load_dataset("Anthropic/hh-rlhf", split=split)
        def add_prompt(example):
            # Use an empty or generic prompt; the preference signal is still usable for DPO-style training.
            example["prompt"] = ""
            return example
        ds = raw.map(add_prompt)
        keep = {"prompt", "chosen", "rejected"}
        drop_cols = [c for c in ds.column_names if c not in keep]
        if drop_cols:
            ds = ds.remove_columns(drop_cols)
        return ds

    raise ValueError(f"Unknown dataset name: {name}")

In [6]:
# Load dataset
train_dataset = load_preference_dataset(DATASET_CHOICE, split="train")

README.md:   0%|          | 0.00/478 [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/930 [00:00<?, ?B/s]

data/train-00000-of-00001-8349d0765e6718(…):   0%|          | 0.00/123M [00:00<?, ?B/s]

data/test-00000-of-00001-ec71e9262143a91(…):   0%|          | 0.00/13.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/112052 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12451 [00:00<?, ? examples/s]

In [7]:
# --- Quick sanity checks & preview ---
print("✅ Loaded preference dataset:")
print("Name          :", DATASET_CHOICE)
print("Num rows      :", len(train_dataset))
print("Columns       :", train_dataset.column_names)

✅ Loaded preference dataset:
Name          : Dahoas/full-hh-rlhf
Num rows      : 112052
Columns       : ['prompt', 'chosen', 'rejected']


In [8]:
# Show a small sample row (truncated)
sample = train_dataset[0]
def preview(text, n=320):
    txt = text if isinstance(text, str) else str(text)
    return (txt[:n] + " [...]") if len(txt) > n else txt

print("\n--- SAMPLE ROW ---")
print("PROMPT  :", preview(sample.get("prompt", "")))
print("CHOSEN  :", preview(sample.get("chosen", "")))
print("REJECTED:", preview(sample.get("rejected", "")))


--- SAMPLE ROW ---
PROMPT  : 

Human: Should you buy a case to protect your cell phone?

Assistant: It depends on your circumstances.  If you carry your phone in a pocket or a purse then you probably want a case.  But if you only need a phone for quick interactions, a case may actually cause more harm than good.  What do you need the phone for?  A [...]
CHOSEN  :  You’re welcome.
REJECTED:  It sounds like you’ve got the basics down.  Any further questions or concerns?  You can send me any feedback you have at help@babba.ai.


In [10]:
def tokenize_pref(example):
    prompt = example["prompt"]
    chosen  = example["chosen"]
    rejected= example["rejected"]
    # Build the input text (you may adjust chat template if needed)
    text_chosen  = f"{prompt}\n\n### Response:\n{chosen}"
    text_rejected= f"{prompt}\n\n### Response:\n{rejected}"
    tokens_chosen   = tokenizer(text_chosen, truncation=True, padding="max_length", max_length=512)
    tokens_rejected = tokenizer(text_rejected, truncation=True, padding="max_length", max_length=512)
    return {
        "input_ids_chosen":   tokens_chosen["input_ids"],
        "attention_mask_chosen":   tokens_chosen["attention_mask"],
        "input_ids_rejected": tokens_rejected["input_ids"],
        "attention_mask_rejected": tokens_rejected["attention_mask"],
    }

tokenized = train_dataset.map(tokenize_pref, batched=False)

Map:   0%|          | 0/112052 [00:00<?, ? examples/s]

In [16]:
from peft import LoraConfig

# Define LoRA configuration
lora_config = LoraConfig(
    r=16, # LoRA attention dimension
    lora_alpha=16, # Alpha parameter for LoRA scaling
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "embed_tokens",
        "lm_head",
    ],
    lora_dropout=0, # Dropout probability for LoRA layers
    bias="none", # Only supports 'none' for now
    task_type="CAUSAL_LM", # Causal Language Modeling
)

dpo_config = DPOConfig(
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    max_grad_norm=1.0,
    num_train_epochs=0.02, # Reduced from 3 to 1 to speed up training
    output_dir="preference_rl_model",
    # you may add more arguments depending on memory & Unsloth usage
)

trainer = DPOTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized,
    args=dpo_config,
    peft_config=lora_config, # Pass the LoRA configuration here
)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:693: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['model.embed_tokens', 'lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


In [17]:
trainer.train()
print("✅ Preference-RL fine-tuning complete!")

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 112,052 | Num Epochs = 1 | Total steps = 141
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 6,475,808 of 140,991,392 (4.59% trained)


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss
1,0.693100,0.000000,0.000000,0.000000,0.000000,-66.244347,-94.539124,4.362129,5.872039,0,0,0
2,0.693100,0.000000,0.000000,0.000000,0.000000,-134.255783,-101.077606,5.385812,5.623065,No Log,No Log,No Log
3,0.694000,-0.000702,0.000989,0.437500,-0.001692,-187.302963,-202.367172,5.599867,5.026332,No Log,No Log,No Log
4,0.696200,-0.004011,0.002134,0.312500,-0.006145,-178.254486,-152.084335,7.134853,6.983817,No Log,No Log,No Log
5,0.691800,0.001203,-0.001514,0.625000,0.002718,-144.415787,-100.030327,6.244908,6.212642,No Log,No Log,No Log
6,0.691500,0.004217,0.000812,0.625000,0.003406,-247.208557,-201.807175,5.927056,7.368699,No Log,No Log,No Log
7,0.693300,0.000102,0.000448,0.500000,-0.000346,-172.504562,-188.156891,7.179503,6.852865,No Log,No Log,No Log
8,0.694500,-0.000433,0.002334,0.375000,-0.002767,-207.298035,-195.831375,5.825998,6.422496,No Log,No Log,No Log
9,0.694400,-0.000554,0.001841,0.562500,-0.002395,-283.063202,-274.951813,6.913712,6.991078,No Log,No Log,No Log
10,0.693600,-0.001422,-0.000618,0.500000,-0.000804,-164.707764,-107.599113,6.081464,5.443498,No Log,No Log,No Log


/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:270: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


✅ Preference-RL fine-tuning complete!


In [19]:
model.save_pretrained("preference_rl_model", safe_serialization=False)
tokenizer.save_pretrained("preference_rl_model")

('preference_rl_model/tokenizer_config.json',
 'preference_rl_model/special_tokens_map.json',
 'preference_rl_model/vocab.json',
 'preference_rl_model/merges.txt',
 'preference_rl_model/added_tokens.json',
 'preference_rl_model/tokenizer.json')

In [20]:
prompt = "What is the difference between data science and machine learning?"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Generated answer:\n", tokenizer.decode(outputs[0], skip_special_tokens=True))

Generated answer:
 What is the difference between data science and machine learning?

